In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('sbux_clv_drop_AOV_and_completed_offers_across_channels.csv')

In [3]:
data

,Unnamed: 0,Customer ID,Age,Gender,HH Income,MemberSince,Recency (# Days ago from last trans. As of final day up to Day 15),Frequency (# Trans. from Day 1-15),Monetary Value (Sum of Trans. from Day 1-15),# of Marketing offers that Starbucks sent to each customer from Day 1-15,# of Marketing offers that were viewed from Day 1-15,# of Marketing offers that were successfully completed from Day 1-15,Marketing Offer View Rate from Day 1-15,Marketing Offer Response Rate from Day 1-15,Sum(Trans. Amt from Day 16-30),CLV_Quantile
0,0,0009655768c64bdeb2e877511632db8f,18-34,M,50k-75k,2017,5,1,22.16,2.0,1.0,0.0,0.50,0.00,105.44,Top 20%
1,1,00116118485d4dfda04fdbaba9a87b5c,Unknown,Unknown,Unknown,2018,2,1,0.70,1.0,1.0,0.0,1.00,0.00,3.39,Bottom 20%
2,2,0011e0d4e6b944f998e987f904e8c1e5,35-50,O,50k-75k,2018,4,2,25.42,3.0,3.0,1.0,1.00,0.33,54.04,40-60%
3,3,0020c2b971eb4e9188eac86d93036a77,51-67,F,75k-100k,2016,8,4,98.33,3.0,1.0,1.0,0.33,0.33,98.53,60-80%
4,4,0020ccbbb6d84e358d3414a3ff76cffd,18-34,F,50k-75k,2016,2,5,62.86,2.0,2.0,1.0,1.00,0.50,91.19,60-80%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15037,15037,fff3ba4757bd42088c044ca26d73817a,68-84,F,75k-100k,2015,1,8,148.84,3.0,2.0,2.0,0.67,0.67,432.14,Top 20%
15038,15038,fff7576017104bcc8677a8d63322b5e1,68-84,M,50k-75k,2017,3,3,16.05,2.0,2.0,1.0,1.00,0.50,13.89,20-40%
15039,15039,fff8957ea8b240a6b5e634b6ee8eafcf,68-84,M,50k-75k,2018,14,1,0.64,0.0,0.0,0.0,0.00,0.00,11.51,20-40%
15040,15040,fffad4f4828548d1b5583907f2e9906b,18-34,M,30k-50k,2017,1,7,42.50,2.0,2.0,1.0,1.00,0.50,46.33,40-60%


In [4]:
# Filter out unknowns for gender, age, and HH income as well as 'O' for gender
data_filtered = data[(data['Gender'] != 'Unknown') & (data['Gender'] != 'O') & 
                     (data['Age'] != 'Unknown') & 
                     (data['HH Income'] != 'Unknown')]

# Find the ratio of each category across the CLV quantiles
gender_ratio = data_filtered.groupby('CLV_Quantile')['Gender'].value_counts(normalize=True)
age_ratio = data_filtered.groupby('CLV_Quantile')['Age'].value_counts(normalize=True)
hh_income_ratio = data_filtered.groupby('CLV_Quantile')['HH Income'].value_counts(normalize=True)

# Round to two decimal places and reorder the indexes
gender_ratio_sorted = gender_ratio.sort_index(level=['CLV_Quantile', 'Gender'], ascending=[True, False]).round(2)
age_ratio_sorted = age_ratio.sort_index(level=['CLV_Quantile', 'Age'], ascending=[True, False]).round(2)
hh_income_ratio_sorted = hh_income_ratio.sort_index(level=['CLV_Quantile', 'HH Income'], ascending=[True, False]).round(2)

gender_ratio_sorted, age_ratio_sorted, hh_income_ratio_sorted

(CLV_Quantile  Gender
 20-40%        M         0.69
               F         0.31
 40-60%        M         0.59
               F         0.41
 60-80%        M         0.49
               F         0.51
 Bottom 20%    M         0.79
               F         0.21
 Top 20%       M         0.46
               F         0.54
 Name: Gender, dtype: float64,
 CLV_Quantile  Age   
 20-40%        85-101    0.03
               68-84     0.16
               51-67     0.34
               35-50     0.26
               18-34     0.22
 40-60%        85-101    0.04
               68-84     0.19
               51-67     0.40
               35-50     0.23
               18-34     0.15
 60-80%        85-101    0.05
               68-84     0.21
               51-67     0.41
               35-50     0.23
               18-34     0.10
 Bottom 20%    85-101    0.03
               68-84     0.14
               51-67     0.29
               35-50     0.25
               18-34     0.28
 Top 20%       85-101    

In [5]:
df=pd.read_csv('sbux_clv_drop_AOV_and_completed_offers_across_channels.csv')
# Exclude rows where Age, Gender, or HHIncome is unknown
df_filtered = df[(df["Age"] != "Unknown") & (df["Gender"] != "Unknown") & (df["HH Income"] != "Unknown")]

# Define the key variables
key_vars = ["Age", "Gender", "HH Income", "Frequency (# Trans. from Day 1-15)", 
            "Monetary Value (Sum of Trans. from Day 1-15)", 
            "# of Marketing offers that Starbucks sent to each customer from Day 1-15", 
            "# of Marketing offers that were viewed from Day 1-15", 
            "# of Marketing offers that were successfully completed from Day 1-15",
            "Marketing Offer View Rate from Day 1-15", 
            "Marketing Offer Response Rate from Day 1-15",
            "Sum(Trans. Amt from Day 16-30)"]

# Create a DataFrame to store the results
summary = pd.DataFrame()

# Calculate the mode for categorical variables and mean for numerical variables for each CLV quantile
for var in key_vars:
    if df[var].dtype == 'object':
        temp = df.groupby("CLV_Quantile")[var].agg(lambda x:x.value_counts().index[0]).reset_index()
    else:
        temp = df.groupby("CLV_Quantile")[var].mean().reset_index()

    summary = pd.concat([summary, temp], axis=1)

# Remove duplicate CLV_Quantile columns
summary = summary.loc[:,~summary.columns.duplicated()]

In [6]:
# Create a DataFrame to store the results
summary_filtered = pd.DataFrame()

# Calculate the mode for categorical variables and mean for numerical variables for each CLV quantile
for var in key_vars:
    if df_filtered[var].dtype == 'object':
        temp = df_filtered.groupby("CLV_Quantile")[var].agg(lambda x:x.value_counts().index[0]).reset_index()
    else:
        temp = df_filtered.groupby("CLV_Quantile")[var].mean().reset_index()

    summary_filtered = pd.concat([summary_filtered, temp], axis=1)

# Remove duplicate CLV_Quantile columns
summary_filtered = summary_filtered.loc[:,~summary_filtered.columns.duplicated()]

summary_filtered

# Set the order for CLV_Quantile
order = ['Bottom 20%', '20-40%', '40-60%', '60-80%', 'Top 20%']

# Convert the CLV_Quantile to a Categorical type with the specified order
summary_filtered['CLV_Quantile'] = pd.Categorical(summary_filtered['CLV_Quantile'], categories=order, ordered=True)

# Sort by CLV_Quantile
summary_filtered = summary_filtered.sort_values(by='CLV_Quantile')


In [7]:
# Round Monetary Value, Frequency, and Sum(Trans. Amt from Day 16-30) to the nearest whole number
summary_filtered["Monetary Value (Sum of Trans. from Day 1-15)"] = summary_filtered["Monetary Value (Sum of Trans. from Day 1-15)"].round(0)
summary_filtered["Frequency (# Trans. from Day 1-15)"] = summary_filtered["Frequency (# Trans. from Day 1-15)"].round(0)
summary_filtered["Sum(Trans. Amt from Day 16-30)"] = summary_filtered["Sum(Trans. Amt from Day 16-30)"].round(0)


# Convert Marketing Offer View Rate and Marketing Offer Response Rate to percentage
summary_filtered["Marketing Offer View Rate from Day 1-15"] = (summary_filtered["Marketing Offer View Rate from Day 1-15"] * 100).round(0)
summary_filtered["Marketing Offer Response Rate from Day 1-15"] = (summary_filtered["Marketing Offer Response Rate from Day 1-15"] * 100).round(0)

summary_filtered=summary_filtered[['CLV_Quantile','Gender', 'HH Income',
       'Frequency (# Trans. from Day 1-15)',
       'Monetary Value (Sum of Trans. from Day 1-15)',
       'Marketing Offer View Rate from Day 1-15',
       'Marketing Offer Response Rate from Day 1-15',
        'Sum(Trans. Amt from Day 16-30)']]

summary_filtered

,CLV_Quantile,Gender,HH Income,Frequency (# Trans. from Day 1-15),Monetary Value (Sum of Trans. from Day 1-15),Marketing Offer View Rate from Day 1-15,Marketing Offer Response Rate from Day 1-15,Sum(Trans. Amt from Day 16-30)
3,Bottom 20%,M,30k-50k,3.0,12.0,58.0,11.0,6.0
0,20-40%,M,50k-75k,4.0,29.0,63.0,24.0,19.0
1,40-60%,M,50k-75k,4.0,53.0,68.0,38.0,40.0
2,60-80%,F,50k-75k,4.0,73.0,71.0,50.0,77.0
4,Top 20%,F,50k-75k,4.0,92.0,72.0,54.0,179.0
